In [1]:
import os
from utils.data_preprocess_tools import (
load_data,
load_ncbi_taxdump,
count_entity,
map_microbe_name2taxid
)
import pandas as pd
import re

## Disbiome Microbial Data

In [2]:
all_microbes = []

In [3]:
disbiome_data_path = os.path.join(os.getcwd(), "json", "disbiome_data.json")

In [4]:
disbiome_data = load_data(disbiome_data_path)
print(f"{len(disbiome_data)} records in Disbiome database")

10866 records in Disbiome database


In [5]:
all_microbes.extend(rec["subject"]["taxid"] for rec in disbiome_data if "taxid" in rec["subject"])
# 95 records have no taxids (10866-10771)
print("Lenth of microbes with taxids:", len(all_microbes))

Lenth of microbes with taxids: 10771


***

## HMDB Microbial Data

In [31]:
hmdb_data_path = os.path.join(os.getcwd(), "json", "hmdb_microbe_metabolites.json")

In [32]:
hmdb_data = load_data(hmdb_data_path)
print(f"{len(hmdb_data)} records in HMDB database")

157 records in HMDB database


In [33]:
microbes = []
no_taxid = []
for rec in hmdb_data:
    if "associated_microbes" in rec:
        for taxon in rec["associated_microbes"]:
            if "taxid" in taxon:
                microbes.append(taxon["taxid"])
            else:
                no_taxid.append(taxon["scientific_name"])

all_microbes.extend(microbes)
print(f"{len(microbes)} microbes with taxids")
print(f"{len(set(no_taxid))} unique microbes with no taxids")
print("Length of microbes with taxids from disbiome and hmdb:", len(all_microbes))

474 microbes with taxids
118 unique microbes with no taxids
Length of microbes with taxids from disbiome and hmdb: 11245


In [34]:
taxdump_path = os.path.join("..", "utils", "taxdump.tar.gz")
mapping_src = load_ncbi_taxdump(taxdump_path)

In [35]:
mapped_microbe_names = map_microbe_name2taxid(
    mapping_src=mapping_src,
    microbe_names=no_taxid,
    unmapped_out_path="manual/012325_hmdb_microbe_names_notfound.csv",
)

count of unique microbial names to map: 118
count of mapped unique microbial names: 61
count of unmapped unique microbial names: 57
Unmapped microbial names saved to: manual/012325_hmdb_microbe_names_notfound.csv


In [36]:
hmdb_filled_taxid = pd.read_csv("manual/012325_hmdb_taxid_filled.csv", sep=",", names=["microbe", "taxid"])
valid_taxids = hmdb_filled_taxid["taxid"].apply(lambda x: x if str(x).isdigit() else None).dropna()
all_microbes.extend(valid_taxids.astype(int).tolist())

print("Length of microbes with taxids from disbiome, hmdb and hmdb filled:", len(all_microbes))
print(f"{len(hmdb_filled_taxid)} microbes with taxids filled from hmdb.")

48 microbes with taxids


In [37]:
all_microbes.extend(hmdb_filled_taxid["taxid"].tolist())
print("Length of microbes with taxids from disbiome, hmdb and hmdb filled:", len(all_microbes))

Length of microbes with taxids from disbiome, hmdb and hmdb filled: 11293


***

## GMMAD2 Microbial Data

In [38]:
gmmad2_data_path = os.path.join(os.getcwd(), "json", "gmmad2_data.json")

In [39]:
gmmad2_data = load_data(gmmad2_data_path)

In [42]:
print(f"{len(gmmad2_data)} records in GMMAD2 database")

1430490 records in GMMAD2 database


In [47]:
microbes = []
unmapped = []
for rec in gmmad2_data:
    if "taxid" in rec["subject"]:
        microbes.append(rec["subject"]["taxid"])
    else:
        if "biolink:Gene" not in rec["object"]["type"]:
            unmapped.append(rec["subject"]["name"])

all_microbes.extend(microbes)
print(f"{len(microbes)} microbes with taxids in gmmad2")
print(f"{len(set(unmapped))} unique microbes with no taxids in gmmad2")

1372498 microbes with taxids in gmmad2
113 unique microbes with no taxids in gmmad2


In [48]:
mapped_microbe_names = map_microbe_name2taxid(
    mapping_src=mapping_src,
    microbe_names=unmapped,
    unmapped_out_path="manual/012325_gmmad2_microbe_names_notfound.csv",
)

count of unique microbial names to map: 113
count of mapped unique microbial names: 0
count of unmapped unique microbial names: 113
Unmapped microbial names saved to: manual/012325_gmmad2_microbe_names_notfound.csv


In [49]:
print(f"{len(all_microbes)} microbes with taxids from disbiome, hmdb and gmmad2")

1383791 microbes with taxids from disbiome, hmdb and gmmad2


In [50]:
unique_microbes = set(all_microbes)
print(f"{len(unique_microbes)} unique microbes with taxids from disbiome, hmdb and gmmad2")

8943 unique microbes with taxids from disbiome, hmdb and gmmad2


## GMMAD2 Gene-Microbe Data

In [86]:
genes_edges = []
genes = []
for rec in gmmad2_data:
    if "biolink:Gene" in rec["object"]["type"]:
        genes_edges.append(rec)
        genes.append(rec["object"]["id"])
print(f"{len(genes_edges)} records with Gene-Metabolite relationship")
print(f"{len(set(genes))} unique genes")

53278 records with Gene-Metabolite relationship
8200 unique genes


In [88]:
# hmdb protein count
proteins = []
for rec in hmdb_data:
    if "associated_proteins" in rec:
        for protein in rec["associated_proteins"]:
            if "name" in protein:
                proteins.append(protein["name"])
print(f"{len(set(proteins))} unique proteins in HMDB database")

535 unique proteins in HMDB database


***

## count unique microbes

In [53]:
valid_microbes = []
invalid_microbes = []

for taxid in unique_microbes:
    try:
        valid_microbes.append(int(taxid))
    except ValueError:
        invalid_microbes.append(taxid)

print("Invalid taxid entries:", invalid_microbes)

Invalid taxid entries: ['taxid']


In [54]:
unique_microbes = [int(taxid) for taxid in valid_microbes]

In [56]:
len(unique_microbes)

8942

In [68]:
import biothings_client as bt
taxon = bt.get_client("taxon")
get_rank = taxon.querymany(unique_microbes, scopes="taxid", fields=["rank", "lineage"])

Input sequence provided is already in string format. No operation performed
40 input query terms found dup hits:	[('33043', 2), ('294', 2), ('303', 2), ('435', 2), ('546', 2), ('1590', 2), ('659', 2), ('662', 2), 
114 input query terms found no hit:	['417', '656024', '1114980', '1803179', '33961', '1551', '67281', '67295', '67334', '67354', '329527


In [70]:
lineage = [lineage for item in get_rank if "notfound" not in item for lineage in item["lineage"]]
unique_lineage = set(lineage)
unique_lineage = [int(lineage) for lineage in unique_lineage]
print(f"{len(unique_lineage)} unique lineages")

12166 unique lineages


In [71]:
get_rank = taxon.querymany(unique_lineage, scopes="taxid", fields=["rank"])
get_rank = {item["query"]: item["rank"] for item in get_rank if "notfound" not in item}
print(f"{len(get_rank)} ranks found")

Input sequence provided is already in string format. No operation performed


12166 ranks found


In [72]:
from collections import Counter
value_counts = Counter(get_rank.values())

print("Occurrences of each value:")
for key, count in value_counts.items():
    print(f"{key}: {count}")

Occurrences of each value:
species: 7208
no rank: 517
superkingdom: 4
genus: 2113
order: 304
family: 679
strain: 822
subphylum: 14
species group: 68
phylum: 72
clade: 68
kingdom: 17
subfamily: 19
subspecies: 51
suborder: 15
infraclass: 2
cohort: 2
class: 137
tribe: 5
subgenus: 5
biotype: 1
serotype: 4
species subgroup: 13
superfamily: 2
subclass: 14
infraorder: 2
section: 3
isolate: 2
subkingdom: 1
serogroup: 1
superorder: 1


## Export data for R analysis

In [73]:
new_rank_dict = {"superkingdom": 4, "kingdom": 17, "phylum": 72, "class": 137, "order": 304, "family": 679, "genus": 2113, "species": 7208, "strain": 822}
old_rank_dict = {"superkingdom": 4, "kingdom": 11, "phylum": 58, "class": 103, "order": 200, "family": 380, "genus": 807, "species": 864}

In [75]:
df = pd.DataFrame({
    "Rank": new_rank_dict.keys(),
    "New_Count": new_rank_dict.values(),
    "Old_Count": [old_rank_dict.get(rank, 0) for rank in new_rank_dict.keys()]
})

In [76]:
csv_file_path = "R_data/rank_comparison.csv"
df.to_csv(csv_file_path, index=False)

In [117]:
old_relation_dict = {"Microbe-Disease": 8469, "Microbe-AnatomicalLocation": 3994, "AnatomicalLocation-Metabolite": 2158, "Gene/Protein-Metabolite": 1010, "Disease-Metabolite": 852, "Pathway-Metabolite": 836, "Microbe-Metabolite": 625}
new_relation_dict = {"Microbe-Disease": 519007, "Microbe-AnatomicalLocation": 3994, "AnatomicalLocation-Metabolite": 2158, "Gene/Protein-Metabolite": 54288, "Disease-Metabolite": 27652, "Pathway-Metabolite": 836, "Microbe-Metabolite": 864982}

In [118]:
df = pd.DataFrame({
    "Relation": new_relation_dict.keys(),
    "New_Count": new_relation_dict.values(),
    "Old_Count": [old_relation_dict.get(relation, 0) for relation in new_relation_dict.keys()]
})

In [119]:
csv_file_path = "R_data/relation_comparison.csv"
df.to_csv(csv_file_path, index=False)

In [123]:
full_relation_dict = {"Microbe-Disease": 519007, "Disease-Metabolite": 27652, "Microbe-Metabolite": 864982}
model_relation_dict = {"Microbe-Disease": 513794, "Disease-Metabolite": 27546, "Microbe-Metabolite": 599801}

In [124]:
df = pd.DataFrame({
    "Relation": full_relation_dict.keys(),
    "Full_Count": full_relation_dict.values(),
    "Model_Count": [model_relation_dict.get(relation, 0) for relation in full_relation_dict.keys()]
})

In [125]:
csv_file_path = "R_data/model_relation_comparison.csv"
df.to_csv(csv_file_path, index=False)

In [114]:
unique_count = {"Microbe":8942, "Disease":898, "Metabolite": 23823, "Gene/Protein": 8735, "Pathway": 385, "AnatomicalLocation": 88}

In [115]:
df = pd.DataFrame({
    "Node": unique_count.keys(),
    "Count": unique_count.values()
})

In [116]:
csv_file_path = "R_data/node_count.csv"
df.to_csv(csv_file_path, index=False)

## count unique pathways

In [90]:
pathways = []
for rec in hmdb_data:
    if "associated_pathways" in rec:
        for pathway in rec["associated_pathways"]:
            if "name" in pathway:
                pathways.append(pathway["name"])
print(f"{len(set(pathways))} unique pathways in HMDB database")

385 unique pathways in HMDB database


## count unique anatomical locations

In [110]:
# disbiome anatomical locations
analocs = [rec["association"]["sources"] for rec in disbiome_data if "sources" in rec["association"]]
print(f"{len(set(analocs))} unique anatomical locations in Disbiome database")

88 unique anatomical locations in Disbiome database


In [111]:
# hmdb anatomical locations
for rec in hmdb_data:
    if "sources" in rec:
        for source in rec["sources"]:
            analocs.extend(source)

filtered_analocs = [loc for loc in set(analocs) if len(loc) > 1]
print(f"{len(set(filtered_analocs))} unique anatomical locations in Disbiome and HMDB database")

88 unique anatomical locations in Disbiome and HMDB database
